In [8]:
import yfinance as yf
import pandas as pd
ticker = yf.Ticker('petr4.sa')
d = ticker.info

In [13]:
d.keys()

dict_values(['20031-912', 'Energy', 49050, 'PetrÃ³leo Brasileiro S.A. - Petrobras produce and sells oil and gas in Brazil and internationally. It engages in prospecting, drilling, refining, processing, trading, and transporting crude oil from producing onshore and offshore oil fields, and shale or other rocks, as well as oil products, natural gas, and other liquid hydrocarbons. The company operates through Exploration and Production; Refining, Transportation and Marketing; Gas and Power; and Corporate and Other Businesses segments. The Exploration and Production segment explores for, develops, and produces crude oil, natural gas liquids, and natural gas primarily for supplies to the its domestic refineries. The Refining, Transportation and Marketing segment engages in the refining, logistics, transport, marketing, and trading of crude oil and oil products; exportation of ethanol; and extraction and processing of shale, as well as holding interests in petrochemical companies. The Gas an